In [26]:
import os
import json
import pandas as pd
import requests
from pyvent.datasets.transactions import ConsumerEdgeSnowflake
from pyvent.tools import SaveTool
from pyvent import ROOT_DIR
from datetime import datetime, timedelta
from pyvent.tools import SaveTool, snowflake_tools
from dateutil.relativedelta import relativedelta


In [27]:
#  List out brands for consumer edge and placer

ce_brands = {'FIRST WATCH RESTAURANTS': 'First Watch',
            'Cracker Barrel': 'Cracker Barrel',
            "Denny's": "Denny's",
            "IHOP": "IHOP",
            "Another Broken Egg": "Another Broken Egg Cafe",
            "Golden Corral": "Golden Corral",
            "Waffle House": "Waffle House",
            "Bob Evans": "Bob Evans",
            "Snooze": "Snooze",
            "Applebee's": "Applebee's",
            "Buffalo Wild Wings": "Buffalo Wild Wings",
            "Cheesecake Factory": "Cheesecake Factory",
            "CHILI'S GRILL & BAR": "Chili's",
            "NINETY NINE RESTAURANTS": "Ninety Nine",
            "Outback Steakhouse": "Outback Steakhouse",
            "Olive Garden": "Olive Garden",
            "Texas Roadhouse": "Texas Roadhouse",
            "TGI Fridays": "TGI Friday's",}

placer_chains = {
    "First Watch": "chain:59b21236173f563be09ae9da",
    "IHOP": "chain:58ad4d484c0da86d5dba923e",
    "Denny's": "chain:59b0312d173f5685147810da",
    "Cracker Barrel": "chain:59b1dbc8173f563bda9ae6b3",
    "Another Broken Egg Cafe": "chain:5b87b8d473a79454429e0b6f",
    "Golden Corral": "chain:59b1d759173f563be49ae6c7",
    "Waffle House": "chain:58acf4ba4c0da86d9ffbc8a3",
    "Bob Evans": "chain:59b1d3bc173f563be29ae611",
    "Snooze": "chain:5b87b8d673a79454429e1282",
    "Applebee's": "chain:59b1cb28173f563bcd9ae611",
    "Buffalo Wild Wings": "chain:59b03be5173f568503781135",
    "Cheesecake Factory": "chain:59b21220173f563bd29aea9e",
    "Chili's": "chain:59b03f78173f5685007811da",
    "Ninety Nine": "chain:59b23354173f563be49aeae4",
    "Outback Steakhouse": "chain:59b1cd2f173f563bea9ae611",
    "Olive Garden": "chain:59b1caa7173f563bd59ae611",
    "Texas Roadhouse": "chain:59b1e314173f563be69ae6c4",
    "TGI Friday's": "chain:59b1dae8173f563bdc9ae6c5",
    # "Cava Grill": "chain:5b41cb6416af4c230fd5ff0e",
    # "Chipotle": "chain:59b031ad173f56851e7810c2",
    # "Five Guys": "chain:59b03b0b173f56850178112c",
    # "Jersey Mike's": "chain:59b03b61173f5685167810f2",
    # "Panera Bread": "chain:59b032f5173f56851c7810f8",
    # "Shake Shack": "chain:5b431f3f1b5c0a67dd394ce8",
    # "Wingstop": "chain:59b04783173f56851778120d",
    # "Zaxby's": "chain:59b1cad2173f563bd19ae611"
}

# map brands to categories, this shows up in the raw output, but I didn't include it in the html
category = {
    "First Watch": "First Watch",
    "IHOP": "Breakfast",
    "Denny's": "Breakfast",
    "Cracker Barrel": "Breakfast",
    "Another Broken Egg Cafe": "Breakfast",
    "Golden Corral": "Breakfast",
    "Waffle House": "Breakfast",
    "Bob Evans": "Breakfast",
    "Snooze": "Breakfast",
    "Applebee's": "Family Dining",
    "Buffalo Wild Wings": "Family Dining",
    "Cheesecake Factory": "Family Dining",
    "Chili's": "Family Dining",
    "Ninety Nine": "Family Dining",
    "Outback Steakhouse": "Family Dining",
    "Olive Garden": "Family Dining",
    "Texas Roadhouse": "Family Dining",
    "TGI Friday's": "Family Dining",
}

income_mapping = {'Less than or equal to $20,000': '<$20k',
                '$20,001-$40,000': '$20-40k',
                '$40,001-$60,000': '$40-60k',
                '$60,001-$80,000': '$60-80k',
                '$80,001-$100,000': '$80-100k',
                '$100,001-$150,000': '$100-150k',
                '$150,001+': '$150k+'}

merch_descs1 = {}
# make all keys in brand1 uppercase
ce_brands = {k.upper(): v for k, v in ce_brands.items()}

brand_mappings = ce_brands.copy()
merch_descs = merch_descs1.copy()
chain_mappings = placer_chains.copy()
category_mappings = category.copy()


In [28]:
def _str_to_sql(s:str) -> str:
    return s.replace("'", r"''")

# get brands from brand_mappings to include in query
query_brands = "'" + "','".join(_str_to_sql(name) for name in brand_mappings.keys()) + "'"
query_brands

"'FIRST WATCH RESTAURANTS','CRACKER BARREL','DENNY''S','IHOP','ANOTHER BROKEN EGG','GOLDEN CORRAL','WAFFLE HOUSE','BOB EVANS','SNOOZE','APPLEBEE''S','BUFFALO WILD WINGS','CHEESECAKE FACTORY','CHILI''S GRILL & BAR','NINETY NINE RESTAURANTS','OUTBACK STEAKHOUSE','OLIVE GARDEN','TEXAS ROADHOUSE','TGI FRIDAYS'"

In [29]:
# connect to snowflake, this will open up a tab on browser
conn = snowflake_tools.get_connection()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://portal.sso.eu-west-1.amazonaws.com/saml/assertion/MDg0NjE4MDA2MjkyX2lucy0xMGU3NGJiODJjNTM3NDY4?SAMLRequest=lZNdc6IwGIX%2FCpO9BgKi1Yza0bJaOwUdxe66d1mINBoSmoSi%2FfUb%2FNhxL9qZvWPCOXlO3pP07w8Fs96JVFTwAfAcCCzCU5FRng%2FAOpnYXWApjXmGmeBkAI5EgfthX%2BGClWhU6Ve%2BJG8VUdoyG3GFmh8DUEmOBFZUIY4LopBO0WoUPSPfgQgrRaQ2OHCxZIoa1qvWJXLduq6duuUImbs%2BhNCFPdeoGsk3cIMov2aUUmiRCna1HMyZPkF4LgwahFEYwuJiHFN%2BHsFXlN9nkUKPSbKwF%2FNVAqzR9XQPgquqIHJF5DtNyXr5fA6gTIKt7Nx1Yc8hlV2b0dmeo7iotwzvSSqKstJmV8d8uVuSuUzk1MxqFg5AuafZW3tSB5u7mi6z8XF6aNXlSxjPcdRe5DE7TL3HnG%2F2ajP5Me6mwHq5Nus3zc6UqsiMN31qswT9wIZdG3YSL0DtDmr7Trvb%2BwWs0ISiHOuT8xq6FFJj5iglbnLjAn8Ijmt1its04%2F7t143CHMa770EUjvxotz%2F%2B9FmVHuEhmq5b8fSJzsOnXZxErTjcBOB8p9Apohz%2B76T67q37cj1j09gsXAhG06M1EbLA%2BvNCPcc7

In [30]:
end_date = datetime.now().strftime("%Y-%m-%d")
start_date = (datetime.now() - relativedelta(years=3) - timedelta(days=datetime.now().weekday())).strftime("%Y-%m-%d")
start_date, end_date

('2021-08-05', '2024-08-06')

In [31]:
def _str_to_sql(s:str) -> str:
    return s.replace("'", r"''")

# get brands from brand_mappings to include in query
query_brands = "'" + "','".join(_str_to_sql(name) for name in brand_mappings.keys()) + "'"

current_freq = 'week'   
# queries consumer edge on snowflake
query = f"""
    SELECT brand.brand_name, date_trunc('{current_freq}', trans.trans_date) as trunc_date, sum(trans.trans_amount) as total_revenue, avg(trans.trans_amount) as AOV, count(trans.transid) as num_orders,
    demographic.income
    FROM consumeredge.public.trans
        LEFT JOIN consumeredge.public.mid_match using(merchid)
        LEFT JOIN consumeredge.public.brand on mid_match.main_brand_id = brand.brand_id
        LEFT JOIN consumeredge.public.demographic on demographic.cardid = trans.cardid
        WHERE brand.brand_name in ({query_brands})
        AND trans.trans_date >= '{start_date}'
        GROUP BY brand.brand_name, trunc_date, demographic.income

"""
cur = conn.cursor()
cur.execute(query)
dfCE = pd.DataFrame(cur.fetchall())
    
dfCE[0] = dfCE[0].map(brand_mappings)

In [32]:
query = f"""
        select 
            card_count / 
            (
            select 
                max(card_count)
            from
                pyvent.consumeredge.panel_size_month
            ) as revn_adj
            , date_trunc('week',trunc_date) as adj_date
        from
            pyvent.consumeredge.panel_size_month
        where
            adj_date >= '{start_date}'
            and adj_date <= '{end_date}'
        order by
            adj_date asc
            
        """
cur = conn.cursor()
cur.execute(query)
df_adj = pd.DataFrame(cur.fetchall())

In [33]:
# convert output datatypes
df_adj[1] = df_adj[1].map(lambda x: x.strftime('%Y-%m-%d'))
df_adj[0] = df_adj[0].astype(float)

dfCE[1] = dfCE[1].map(lambda x: x.strftime('%Y-%m-%d'))

In [34]:
# convert monthly adjustment factor to weekly by taking value from last update
week_dates = pd.date_range(start=datetime.strptime(start_date, "%Y-%m-%d") - timedelta(days=7),end=datetime.now(),freq='w-mon')
week_dates = pd.DataFrame(week_dates.strftime('%Y-%m-%d'),columns=["Date"])

rev_adj = week_dates.merge(df_adj, left_on='Date', right_on=1, how='left')
rev_adj = rev_adj.fillna(method='ffill').fillna(method='bfill').drop(columns=1).rename(columns={0:'rev_adj'})

# Scale total revenue and num_orders by adjustment factor
dfCE = dfCE.merge(rev_adj, left_on=1, right_on='Date', how='left')

dfCE[2] = dfCE[2] / dfCE['rev_adj']
dfCE[4] = dfCE[4] / dfCE['rev_adj']


In [35]:
# split into different dataframes
dfTotalRev = dfCE.pivot_table(index=0, columns=1, values=2, aggfunc='sum', dropna=False)
dfTotalRev.index.name = None
dfAOV = dfCE.pivot_table(index=0, columns=1, values=3, aggfunc='sum', dropna=False)
dfAOV.index.name = None
dfNumOrders = dfCE.pivot_table(index=0, columns=1, values=4, aggfunc='sum', dropna=False)
dfNumOrders.index.name = None

In [36]:
dfOrderDemo = dfCE.copy()
dfOrderDemo[0] = dfOrderDemo[0].map(category_mappings)
dfOrderDemo[5] = dfOrderDemo[5].map(income_mapping)
dfOrderDemo = dfOrderDemo.pivot_table(index=[0,5], columns=1, values=4, aggfunc='sum', dropna=False)
dfOrderDemo.index.name = None

In [37]:
# query placer api
url = "https://papi.placer.ai/v1/reports/visit-trends"


headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "x-api-key": "qsjb4zPcw42I17paw32A78utozd6a6J96Hc3cypT"
}

# write code to get the data for each chain
df = pd.DataFrame()
for chain in chain_mappings:
    print(chain)
    payload = {
            "apiIds": [chain_mappings[chain]],
            "startDate": start_date,
            "endDate": end_date,
            "granularity": 'week',
        }

    response = requests.post(url, headers=headers, data=json.dumps(payload))
    data = response.json()
    # add to different sheets of the same excel file
    df = df.append(pd.DataFrame(data['data'][0]))


First Watch


C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\3727466310.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data['data'][0]))


IHOP


C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\3727466310.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data['data'][0]))


Denny's


...

C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\3727466310.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data['data'][0]))


Texas Roadhouse


C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\3727466310.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data['data'][0]))


TGI Friday's


C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\3727466310.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame(data['data'][0]))


In [38]:
dfPlacer = df.copy()

# pivot so dates are columns
dfPlacer = dfPlacer.pivot(index='apiId', columns='dates', values=['visits', 'panelVisits'])
# replace apiId with chain name
dfPlacer.index = dfPlacer.index.map({v: k for k, v in chain_mappings.items()})
dfPlacer.index.name = None

dfVisits = pd.DataFrame(index=dfPlacer.index)
dfPanelVisits = pd.DataFrame(index=dfPlacer.index)
# reformatting into visits and panel visits
for col in dfPlacer.columns:
    if col[0] == 'visits':
        dfVisits.loc[:, col[1]] = dfPlacer[col]
    else:
        dfPanelVisits.loc[:, col[1]] = dfPlacer[col]
        
# add column for category based on category_mappings
dfVisits['category'] = dfVisits.index.map(category_mappings)
dfPanelVisits['category'] = dfPanelVisits.index.map(category_mappings)
dfNumOrders['category'] = dfNumOrders.index.map(category_mappings)
dfTotalRev['category'] = dfTotalRev.index.map(category_mappings)
dfAOV['category'] = dfAOV.index.map(category_mappings)

# set index to be category and brand
dfVisits.set_index('category', append=True, inplace=True)
dfPanelVisits.set_index('category', append=True, inplace=True)
dfNumOrders.set_index('category', append=True, inplace=True)
dfTotalRev.set_index('category', append=True, inplace=True)
dfAOV.set_index('category', append=True, inplace=True)

# reorder and sort index so first watch is always first, then breakfast, then family dining
dfVisits = dfVisits.reorder_levels([1, 0])
dfVisits.sort_index(inplace=True)
dfVisits = dfVisits.reindex(index=['First Watch', 'Breakfast', 'Family Dining'], level=0)

dfPanelVisits = dfPanelVisits.reorder_levels([1, 0])
dfPanelVisits.sort_index(inplace=True)
dfPanelVisits = dfPanelVisits.reindex(index=['First Watch', 'Breakfast', 'Family Dining'], level=0)

dfNumOrders = dfNumOrders.reorder_levels([1, 0])
dfNumOrders.sort_index(inplace=True)
dfNumOrders = dfNumOrders.reindex(index=['First Watch', 'Breakfast', 'Family Dining'], level=0)

dfTotalRev = dfTotalRev.reorder_levels([1, 0])
dfTotalRev.sort_index(inplace=True)
dfTotalRev = dfTotalRev.reindex(index=['First Watch', 'Breakfast', 'Family Dining'], level=0)

dfAOV = dfAOV.reorder_levels([1, 0])
dfAOV.sort_index(inplace=True)
dfAOV = dfAOV.reindex(index=['First Watch', 'Breakfast', 'Family Dining'], level=0)

index = pd.MultiIndex.from_product([['First Watch', 'Breakfast', 'Family Dining'], list(income_mapping.values())[::-1]], names=['Category', 'Income'])
dfOrderDemo = dfOrderDemo.reindex(index=index)

C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\1345876020.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfVisits.loc[:, col[1]] = dfPlacer[col]
C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\1345876020.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  dfVisits.loc[:, col[1]] = dfPlacer[col]
C:\Users\ZWayne\AppData\Local\Temp\ipykernel_12564\1345876020.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider

In [40]:
# take dfTotalRev and make a YoY grown df for the last year of weeks compared to the previous year of weeks
dfTotalRevYoY = dfTotalRev.copy()
# get the last year of weeks, i added a buffer in case not all weeks are in yet
last_year = dfTotalRevYoY.columns[-53:-1]
# get the previous year of weeks
prev_year = dfTotalRevYoY.columns[-105:-53]
# get the YoY growth
dfTotalRevYoY = dfTotalRevYoY[last_year].div(dfTotalRevYoY[prev_year].values)
dfTotalRevYoY = dfTotalRevYoY - 1
dfTotalRevYoY = dfTotalRevYoY 
dfTotalRevYoY = dfTotalRevYoY.round(2)

dfAOVYoY = dfAOV.copy()
dfAOVYoY = dfAOVYoY[last_year].div(dfAOVYoY[prev_year].values)
dfAOVYoY = dfAOVYoY - 1
dfAOVYoY = dfAOVYoY 
dfAOVYoY = dfAOVYoY.round(2)

dfNumOrdersYoY = dfNumOrders.copy()
dfNumOrdersYoY = dfNumOrdersYoY[last_year].div(dfNumOrdersYoY[prev_year].values)
dfNumOrdersYoY = dfNumOrdersYoY - 1
dfNumOrdersYoY = dfNumOrdersYoY
dfNumOrdersYoY = dfNumOrdersYoY.round(2)

dfOrderDemoYoY = dfOrderDemo.copy()
dfOrderDemoYoY = dfOrderDemoYoY[last_year].div(dfOrderDemoYoY[prev_year].values)
dfOrderDemoYoY = dfOrderDemoYoY - 1
dfOrderDemoYoY = dfOrderDemoYoY
dfOrderDemoYoY = dfOrderDemoYoY.round(2)

dfVisitsYoY = dfVisits.copy()
last_year = dfVisitsYoY.columns[-52:]
prev_year = dfVisitsYoY.columns[-104:-52]
dfVisitsYoY = dfVisitsYoY[last_year].div(dfVisitsYoY[prev_year].values)
dfVisitsYoY = dfVisitsYoY - 1
dfVisitsYoY = dfVisitsYoY
dfVisitsYoY = dfVisitsYoY.round(2)

dfPanelVisitsYoY = dfPanelVisits.copy()
dfPanelVisitsYoY = dfPanelVisitsYoY[last_year].div(dfPanelVisitsYoY[prev_year].values)
dfPanelVisitsYoY = dfPanelVisitsYoY - 1
dfPanelVisitsYoY = dfPanelVisitsYoY 
dfPanelVisitsYoY = dfPanelVisitsYoY.round(2)

In [53]:
# write all dataframes to excel, putting YoY growth below the original data for each sheet
with pd.ExcelWriter('output.xlsx') as writer:
    dfTotalRev.to_excel(writer, sheet_name='Total Revenue', startrow=0)
    dfTotalRevYoY.to_excel(writer, sheet_name='Total Revenue', startrow=dfTotalRev.shape[0] + 2)
    writer.sheets['Total Revenue'].freeze_panes(0, 2)
    dfAOV.to_excel(writer, sheet_name='AOV', startrow=0)
    dfAOVYoY.to_excel(writer, sheet_name='AOV', startrow=dfAOV.shape[0] + 2)
    writer.sheets['AOV'].freeze_panes(0, 2)
    dfNumOrders.to_excel(writer, sheet_name='Number of Transactions', startrow=0)
    dfNumOrdersYoY.to_excel(writer, sheet_name='Number of Transactions', startrow=dfNumOrders.shape[0] + 2)
    writer.sheets['Number of Transactions'].freeze_panes(0, 2)
    dfVisits.to_excel(writer, sheet_name='Visits', startrow=0)
    dfVisitsYoY.to_excel(writer, sheet_name='Visits', startrow=dfVisits.shape[0] + 2)
    writer.sheets['Visits'].freeze_panes(0, 2)
    # dfPanelVisits.to_excel(writer, sheet_name='Est. Same-Store Visits', startrow=0)
    # dfPanelVisitsYoY.to_excel(writer, sheet_name='Est. Same-Store Visits', startrow=dfPanelVisits.shape[0] + 2)
    # writer.sheets['Est. Same-Store Visits'].freeze_panes(0, 2)
    dfOrderDemo.to_excel(writer, sheet_name='Number of Orders by Income', startrow=0)
    dfOrderDemoYoY.to_excel(writer, sheet_name='Number of Orders by Income', startrow=dfOrderDemo.shape[0] + 2)
    writer.sheets['Number of Orders by Income'].freeze_panes(0, 2)